In [168]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [169]:
tf.enable_eager_execution()

import numpy as np
import os
import time
import requests, re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd

Возьмем какие-нибудь сырые данные и сделаем модель для генерации текста. 

Спойлер: я построил модель где в качестве токенов использовал слова, а не символы, хотя получившаяся модель на символах у меня генериала отдельные слова (именно слова а не белеберду) ее обучение на моем компьютере занимало слишком много времени, полтора часа на одну эпоху, поэтому использовать в качестве токенизации слова принесло мне больший импрув, такой как ускорение обучения за счет уменьшения обрабатывемой последовательности (timestamp) в GRU и более осмысленный текст на выходе. 

В качестве корпуса текста я собрал готовые сочинения по произведению Пушкина "Евгений Онегин" с сайта litra.ru

Сочинения по произведению "Евгений Онегин"- http://www.litra.ru/composition/work/woid/00028601184773070301/

In [170]:
general_url = 'http://www.litra.ru/composition/work/woid/00028601184773070301/'

sess = requests.Session()
g = sess.get(general_url)
b=BeautifulSoup(g.text, "html.parser")
element = b.find('div', {'id': 'child_id2'})
urles = re.findall('<a href=[^>]+', str(element))

urles = ['http://www.litra.ru' + x.replace('<a href="', '').replace('"', '') for x in urles]

In [171]:
urles[:5]

['http://www.litra.ru/composition/get/coid/00160151213701291695/woid/00028601184773070301/',
 'http://www.litra.ru/composition/get/coid/00853061230017597355/woid/00028601184773070301/',
 'http://www.litra.ru/composition/get/coid/00061101184864166554/woid/00028601184773070301/',
 'http://www.litra.ru/composition/get/coid/00043801184864193473/woid/00028601184773070301/',
 'http://www.litra.ru/composition/get/coid/00067801184864021931/woid/00028601184773070301/']

Выгрузим сочинения по полученным ссылкам:

In [172]:
texts = []

for i, url in tqdm_notebook(enumerate(urles), total = len(urles)):
    g = sess.get(url)
    b=BeautifulSoup(g.text, "html.parser")

    tags_list = b.find_all('p', {'align':'justify'})
    
    text = re.sub('[^а-яА-Я0-9a-zA-z;?!:,. -/s]', '', tags_list[0].text)
    text = re.sub('[-/s;:,.?!""()]', ' ', text)
    text = re.sub('[ ]+', ' ', text)
    text = text.lower()
    text = text.strip(' ')
    
    texts.append(text)
    with open('reviews/review_' + ('%03d' % i) + '.txt', 'w') as f:
        f.write(text)
        
    time.sleep(1) # чтобы не долбиться слишком часто на сервер

Загрузка текстов

In [173]:
texts = []

for i in range(306):
    with open('reviews/review_' + ('%03d' % i) + '.txt', 'r') as file:
        texts.append(file.read())
        
text = ' '.join(texts)

In [174]:
print(texts[0][:110]+ '...')
print(len(texts))

евгений онегин , пожалуй , самое трудное для понимания произведение русской литературы . про него не скажешь с...
306


In [175]:
text_words = text.split(' ')

print(len(text_words))
print(len(set(text_words)))

249939
25158


$\textbf{TextProcessing}$

Перед обучением модели, нам нужно сопоставить строковое представление численному. Для этого создадим два словаря сопоставляющих слова числам и наоборот

In [176]:
vocab = sorted(set(text_words))

In [177]:
# mapping from unique words to indxs
word2idx = {u: v for v, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in text_words])

Теперь мы получили численное представление слов. Заметим что мы отобразили каждое слово в его позицию от 0 до len(unique)

In [178]:
print ('{} ---- words mapped to int ---- > {}'.format(text_words[:3], text_as_int[:3]))

print ('{} ---- words mapped to int ---- > {}'.format(' '.join(map(str, text_as_int[:3])),
                                                      ' '.join(map(lambda x: idx2word[x],
                                                                   text_as_int[:3]))))

['евгений', 'онегин', ','] ---- words mapped to int ---- > [ 4998 12337     5]
4998 12337 5 ---- words mapped to int ---- > евгений онегин ,


$\textbf{The prediction task}$

Пусть у нас есть слово, или последоваьтельность слов, какое наиболее вероятное следующее слово? Это та задача для решения которой мы тренируем модель. Вход в модель есть последовательность слов, и мы тренируем модель чтобы на выходе модели получить предсказание следующего слова на каждом шаге.

Разделим текст на обучающие примеры и таргеты. Каждый обучающий пример содержит последовательность слов текста. Соответствующие таргеты включают в себя последовательность такой же длины, кроме одного сдвинутого по тексту слова.

In [179]:
seq_length = 6

In [180]:
chunks = tf.data.Dataset.from_tensor_slices(text_as_int).batch(seq_length+1, drop_remainder=True)

for item in chunks.take(5):
    print(repr(' '.join(idx2word[item.numpy()])))

'евгений онегин , пожалуй , самое трудное'
'для понимания произведение русской литературы . про'
'него не скажешь словами твардовского вот стихи'
', а все понятно , все на'
'русском языке . язык этого романа в'


Далее, создадим входную последовательность и таргет для всех текстов:

In [181]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = chunks.map(split_input_target)

Мы используем tf.data чтобы поделить текст на куски и разбить по секциям. Но перед тем как скормить эти данные в модель, нам нужно перемешать их и упаковать их в батчи.

In [184]:
# Batch size 
BATCH_SIZE = 40
BUFFER_SIZE = 10000

# dataset = dataset.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Воспользуемся tf.keras API для создания модели и ее кастомизации так, как мы захотим. Мы определим четыре слоя для определения нашей модели:

 - Слой Эмбеддингов: тренируемая матрица которая отображает числа для каждого символа в вектор с размерностью embedding_dim;
 - GRU слой: тип RNN модели с layer size = units.
 - Dropout слой: регуляризация сети за счет зануления узлов сети с вероятностями rate
 - Dense слой: полносвязный слой c числом выходов равным vocab_size

In [185]:
class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, rate):
        super(Model, self).__init__()
        self.units = units
        self.rate = rate

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru = tf.keras.layers.GRU(self.units, 
                                       return_sequences=True, 
                                       recurrent_activation='sigmoid', 
                                       recurrent_initializer='glorot_uniform', 
                                       stateful=True)
        
        self.dropout = tf.keras.layers.Dropout(self.rate)
        
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        embedding = self.embedding(x)

        # output at every time step
        # output shape == (batch_size, seq_length, hidden_size) 
        output = self.gru(embedding)
        
        dropout_output = self.dropout(output)

        # The dense layer will output predictions for every time_steps(seq_length)
        # output shape after the dense layer == (seq_length * batch_size, vocab_size)
        prediction = self.fc(dropout_output)

        # states will be used to pass at every step to the model while training
        return prediction

In [186]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 300

# Number of RNN units
units = 1024

rate = 0.5

model = Model(vocab_size, embedding_dim, units, rate)

In [187]:
# Using adam optimizer with default arguments
optimizer = tf.train.AdamOptimizer()

# Using sparse_softmax_cross_entropy so that we don't have to create one-hot vectors
def loss_function(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels=real, logits=preds)

In [162]:
model.build(tf.TensorShape([BATCH_SIZE, seq_length]))

In [163]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "word_generating")

In [164]:
EPOCHS = 10

In [165]:
from tqdm import tqdm_notebook

In [166]:
# Training loop
for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()
    
    for (batch, (inp, target)) in tqdm_notebook(enumerate(dataset)):
        with tf.GradientTape() as tape:
            # feeding the hidden state back into the model
            # This is the interesting step
            predictions = model(inp)
            loss = loss_function(target, predictions)
              
            grads = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(zip(grads, model.variables))

            if batch % 100 == 0:
                print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,
                                                            batch,
                                                            loss))
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix)

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 10.1329
Epoch 1 Batch 100 Loss 7.8617
Epoch 1 Batch 200 Loss 7.4258
Epoch 1 Batch 300 Loss 7.5139
Epoch 1 Batch 400 Loss 7.0018
Epoch 1 Batch 500 Loss 6.5758
Epoch 1 Batch 600 Loss 7.1284
Epoch 1 Batch 700 Loss 7.0770
Epoch 1 Batch 800 Loss 6.9369

Epoch 1 Loss 6.7458
Time taken for 1 epoch 605.5641601085663 sec



Epoch 2 Batch 0 Loss 7.4648
Epoch 2 Batch 100 Loss 6.4731
Epoch 2 Batch 200 Loss 6.3311
Epoch 2 Batch 300 Loss 6.5786
Epoch 2 Batch 400 Loss 6.2389
Epoch 2 Batch 500 Loss 6.1548
Epoch 2 Batch 600 Loss 6.0610
Epoch 2 Batch 700 Loss 5.9246
Epoch 2 Batch 800 Loss 6.1556

Epoch 2 Loss 6.1654
Time taken for 1 epoch 12380.206770181656 sec



Epoch 3 Batch 0 Loss 6.2223
Epoch 3 Batch 100 Loss 5.9320
Epoch 3 Batch 200 Loss 5.7796
Epoch 3 Batch 300 Loss 5.5128
Epoch 3 Batch 400 Loss 5.5671
Epoch 3 Batch 500 Loss 6.0906
Epoch 3 Batch 600 Loss 5.5489
Epoch 3 Batch 700 Loss 5.0239
Epoch 3 Batch 800 Loss 5.7456

Epoch 3 Loss 5.7142
Time taken for 1 epoch 2279.252575159073 sec



Epoch 4 Batch 0 Loss 5.3263
Epoch 4 Batch 100 Loss 4.9595
Epoch 4 Batch 200 Loss 4.7749
Epoch 4 Batch 300 Loss 5.0907
Epoch 4 Batch 400 Loss 4.8068
Epoch 4 Batch 500 Loss 4.4789
Epoch 4 Batch 600 Loss 5.0804
Epoch 4 Batch 700 Loss 4.8896
Epoch 4 Batch 800 Loss 4.9269

Epoch 4 Loss 4.7424
Time taken for 1 epoch 3216.463793992996 sec



Epoch 5 Batch 0 Loss 4.4049
Epoch 5 Batch 100 Loss 3.9053
Epoch 5 Batch 200 Loss 4.1232
Epoch 5 Batch 300 Loss 4.6333
Epoch 5 Batch 400 Loss 4.8276
Epoch 5 Batch 500 Loss 4.2092
Epoch 5 Batch 600 Loss 4.1084
Epoch 5 Batch 700 Loss 3.8712
Epoch 5 Batch 800 Loss 4.3633

Epoch 5 Loss 3.9693
Time taken for 1 epoch 3421.8115670681 sec



Epoch 6 Batch 0 Loss 3.9793
Epoch 6 Batch 100 Loss 3.8289
Epoch 6 Batch 200 Loss 4.0254
Epoch 6 Batch 300 Loss 3.8040
Epoch 6 Batch 400 Loss 3.7119
Epoch 6 Batch 500 Loss 3.8590
Epoch 6 Batch 600 Loss 3.4516
Epoch 6 Batch 700 Loss 3.9109
Epoch 6 Batch 800 Loss 3.6048

Epoch 6 Loss 4.0384
Time taken for 1 epoch 3422.079575061798 sec



Epoch 7 Batch 0 Loss 3.2844
Epoch 7 Batch 100 Loss 3.2995
Epoch 7 Batch 200 Loss 3.2825
Epoch 7 Batch 300 Loss 3.2983
Epoch 7 Batch 400 Loss 3.3775
Epoch 7 Batch 500 Loss 3.1249
Epoch 7 Batch 600 Loss 3.7902
Epoch 7 Batch 700 Loss 3.3338
Epoch 7 Batch 800 Loss 3.2933

Epoch 7 Loss 3.4825
Time taken for 1 epoch 1697.2256598472595 sec



Epoch 8 Batch 0 Loss 3.1688
Epoch 8 Batch 100 Loss 3.4133
Epoch 8 Batch 200 Loss 3.1844
Epoch 8 Batch 300 Loss 3.0808
Epoch 8 Batch 400 Loss 3.2442
Epoch 8 Batch 500 Loss 3.1951
Epoch 8 Batch 600 Loss 3.0616
Epoch 8 Batch 700 Loss 3.1360
Epoch 8 Batch 800 Loss 3.0253

Epoch 8 Loss 3.1729
Time taken for 1 epoch 606.1038897037506 sec



Epoch 9 Batch 0 Loss 2.7709
Epoch 9 Batch 100 Loss 2.8727
Epoch 9 Batch 200 Loss 2.9146
Epoch 9 Batch 300 Loss 2.9175
Epoch 9 Batch 400 Loss 2.8920
Epoch 9 Batch 500 Loss 3.0812
Epoch 9 Batch 600 Loss 2.9267
Epoch 9 Batch 700 Loss 2.9948
Epoch 9 Batch 800 Loss 2.7045

Epoch 9 Loss 2.8344
Time taken for 1 epoch 608.6046841144562 sec



Epoch 10 Batch 0 Loss 2.6902
Epoch 10 Batch 100 Loss 2.5019
Epoch 10 Batch 200 Loss 2.4867
Epoch 10 Batch 300 Loss 3.0782
Epoch 10 Batch 400 Loss 2.7141
Epoch 10 Batch 500 Loss 2.6382
Epoch 10 Batch 600 Loss 2.5876
Epoch 10 Batch 700 Loss 2.5639
Epoch 10 Batch 800 Loss 2.8332

Epoch 10 Loss 2.7728
Time taken for 1 epoch 610.4138381481171 sec



In [144]:
# Evaluation step (generating text using the learned model)

# Number of characters to generate
num_generate = 200

# You can change the start string to experiment
start_string = 'роман'

# Converting our start string to numbers (vectorizing) 
input_eval = [word2idx[s] for s in [start_string]*BATCH_SIZE]
input_eval = tf.expand_dims(input_eval, 1)

# Empty string to store our results
text_generated = []

# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.
temperature = 1.0

In [145]:
# Evaluation loop.

# Here batch size == 1
model.reset_states()
for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.expand_dims(tf.squeeze(predictions[0], 0), 0)

    # using a multinomial distribution to predict the word returned by the model
    predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
    
    # We pass the predicted word as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id]*BATCH_SIZE, 1)
    
    text_generated.append(idx2word[predicted_id])

print (start_string + ' ' + ' '.join(text_generated))

роман явился энциклопедией русской жизни ведь в этой энциклопедии заметное место заняли также описания природы которые появляются на страницах романа в стихах была поставлена проблема отсутствия достойного поля прохлада сумрачной дубравы журчанье тихого ручья на бумаге невидимые законы по своим размерам и крайне разнообразная по словам белинского а белинский татьяна хотела узнать питает ли онегин он холоден спокоен но обратимся к 8 главе он пишет покамест упивайтесь он старался сдержать свою иронию пушкин пишет ей душно здесь она мечтой стремится к жизни полевой поместный круг татьяна один день идеально вкусу наряды глубиной содержания натуры е душу и терзался оттого что никогда не были онегин это человек решивший построить свою реальную жизнь по законам европейского романа он написал но мало места уделялось чувствам пушкин работал над романом пушкин был своему приятелю наставником покровителем любовь но не настолько у тетушки княжны елены все тот же тюлевый чепец познакомившись ни пот

# Итог

Я использовал генерацию по словам, а не по символам потому что обучение на последовательностях длины 100 занимает очень много времени, около полуторачаса на эпоху. В итоге видно что текст адекватно генерится периодами, то есть слова связаны по смыслу в пределах 3-4 слов, затем начинается уже новая смысловая фраза такой же длины. Думаю чтобы это побороть нужно в процессе обучения брать более длинную последовательность. 

Базовую модель я взял из гайда тензорфлоу и добавил в обучение Dropout слой. Весь вышенаписанный код я реализовал в виде python модуля. Который скачивает данные с Litra.ru или берет уже из существующей папки с сочинениями, обучает и предиктит.